In [1]:
def rel_err(val: float, true_val: float) -> float:
    return abs(val - true_val) / abs(true_val)

 
How to approximate the natural logarithm of any real number using the Taylor series expansion and some logarithmic properties?

In [2]:
from math import log, e
import plotly.graph_objects as go


class Log:
    def __init__(self, 
            a_tol: float = 1e-12,
            _pivot: float = e**0.1,
            _log_pivot: float | None = 0.1
        ):
        """
        Class for numerical approximation of a natural logarithm of any real number.
        """
        assert 1 < _pivot < 2
        self._pivot = _pivot
        self._a_tol = a_tol
        self._log_pivot = self.taylor(_pivot) if _log_pivot is None else _log_pivot
    
    def taylor(self, x: float) -> float:
        assert 1 < x <= self._pivot
        x -= 1
        _x = x
        s = 0; old = self._a_tol; i = 1
        sign = 1
        while abs(s - old) >= self._a_tol:
            old = s
            s += sign * x / i
            sign *= -1; x *= _x; i += 1
        return s
    
    def __call__(self, x: float) -> float:
        pow = 0
        while x > self._pivot:
            x /= self._pivot
            pow += 1
        while x < 1:
            x *= self._pivot
            pow -= 1
        res = self.taylor(x) if x != self._pivot else self._log_pivot
        return res + pow * self._log_pivot
    
    def evaluate(self) -> ...:
        self.vals = [
            list(range(2, 1000)),
            [2**-el for el in range(1, 20)],
            [1+el for el in [2**-el for el in range(1, 20)]],
            [1-el for el in [2**-el for el in range(1, 20)]],
            [e**el for el in range(1, 20)]
        ]
        self.errs = [[rel_err(self(el), log(el)) for el in sublist] for sublist in self.vals]
        errors_min_max = [(min(el), max(el)) for el in self.errs]
        return errors_min_max
    
    def plot(self):
        fig = go.Figure()

        for i, (val, err) in enumerate(zip(self.vals, self.errs)):
            fig.add_trace(go.Scatter(x=val, y=err, name=f'err{i}'))
        fig.update_xaxes(type='log')
        fig.update_yaxes(type='log')
        fig.update_layout(template='plotly_dark')
        return fig

In [3]:
my_ln = Log()

In [4]:
my_ln.evaluate()

[(0.0, 8.216788982285264e-14),
 (2.84749022561326e-16, 9.152647153756908e-15),
 (0.0, 4.547751058974256e-13),
 (8.008566259537295e-16, 7.0128038665458895e-09),
 (1.3816108750890837e-15, 1.4210854715202004e-14)]

In [5]:
my_ln.plot()

In [6]:
log3_2_my = my_ln(3) / my_ln(2); log3_2_true = log(3, 2)
log3_2_my, log3_2_true, rel_err(log3_2_my, log3_2_true)

(1.5849625007210082, 1.5849625007211563, 9.344306342743682e-14)